In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 28.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-man

In [ ]:
import spacy
from sklearn.metrics.pairwise import manhattan_distances
import requests
from requests.structures import CaseInsensitiveDict
import time

In [ ]:
from google.colab import userdata

api_key = userdata.get('GEOAPIFY_KEY')

In [ ]:
ner = spacy.load("en_core_web_trf")

In [ ]:
coordinates = {
    'Delhi': [77.1025, 28.7041],
    'Gurgaon': [28.459967, 77.394535],
    'Bangalore': [12.9716, 77.5946],
    'Assam': [26.2006, 92.9378],
    'New York': [40.7128, -74.0060],
    'India': [20.5937, 78.9629],
}

In [ ]:
path = '/content/drive/MyDrive/ISRO Hackathon/hypothetical_flood_Data.txt'

with open(path) as f:
  data = f.readlines()

data = [x.strip() for x in data if x.strip() != ""]

In [ ]:
for d in data:
  for i in d.split(". "):
    print(i)
  print()

In the early monsoon season of 2023, Delhi experienced one of the most severe floods in recent history
Heavy rainfall, starting in late June and continuing for almost a week, led to the overflowing of the Yamuna River, which breached its banks and inundated vast areas of the city
The relentless downpour resulted in accumulated water levels that quickly overwhelmed the drainage systems, causing widespread flooding in numerous parts of the city.

Residential neighborhoods such as Mayur Vihar, Okhla, and parts of East Delhi were particularly hard-hit, with homes and apartment complexes submerged under several feet of water
The historic areas around Chandni Chowk and Connaught Place also saw significant water accumulation, affecting businesses and leading to substantial economic losses.

Commercial districts like Nehru Place and Connaught Place, which house numerous offices and retail establishments, faced severe disruptions as floodwaters entered buildings, damaging property and halting o

In [ ]:
len(data)

61

In [ ]:
def get_coordinates(query):
  url = f"https://api.geoapify.com/v1/geocode/search?text={query}&lang=en&limit=3&filter=countrycode:in&format=json&apiKey={api_key}"

  headers = CaseInsensitiveDict()
  headers["Accept"] = "application/json"

  resp = requests.get(url, headers=headers)

  return resp.json()['results']

In [ ]:
get_coordinates("Gurgaon, Haryana")

[{'datasource': {'sourcename': 'openstreetmap',
   'attribution': '© OpenStreetMap contributors',
   'license': 'Open Database License',
   'url': 'https://www.openstreetmap.org/copyright'},
  'country': 'India',
  'country_code': 'in',
  'state': 'Haryana',
  'city': 'Gurugram District',
  'district': 'Gurgaon',
  'lon': 77.00270014657752,
  'lat': 28.42826235,
  'state_code': 'HR',
  'result_type': 'district',
  'formatted': 'Gurgaon, Gurugram District, HR, India',
  'address_line1': 'Gurgaon',
  'address_line2': 'Gurugram District, HR, India',
  'category': 'administrative',
  'timezone': {'name': 'Asia/Kolkata',
   'offset_STD': '+05:30',
   'offset_STD_seconds': 19800,
   'offset_DST': '+05:30',
   'offset_DST_seconds': 19800,
   'abbreviation_STD': 'IST',
   'abbreviation_DST': 'IST'},
  'plus_code': '7JWVC2H3+83',
  'rank': {'importance': 0.65001,
   'popularity': 3.6742540863607367,
   'confidence': 1,
   'confidence_city_level': 1,
   'match_type': 'full_match'},
  'place_id':

In [ ]:
coord = []
count = 0

for query in data:
  doc = ner(query)
  for ent in doc.ents:
    if ent.label_ == "GPE" or ent.label_ == "LOC":
      res = get_coordinates(ent.text)
      for r in res:
        arr = [r['lon'], r['lat']]
        coord.append([query, arr])

  count += 1
  print(count)

  time.sleep(1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [ ]:
len(coord)

91

In [ ]:
target = coordinates['Delhi']

res = []

for loc in coord:
  dist = manhattan_distances([target], [loc[1]])

  if dist < 5:
    if loc[0] not in res:
      res.append(loc[0])

In [ ]:
for r in res:
  print(r)
  print()

In the early monsoon season of 2023, Delhi experienced one of the most severe floods in recent history. Heavy rainfall, starting in late June and continuing for almost a week, led to the overflowing of the Yamuna River, which breached its banks and inundated vast areas of the city. The relentless downpour resulted in accumulated water levels that quickly overwhelmed the drainage systems, causing widespread flooding in numerous parts of the city.

Residential neighborhoods such as Mayur Vihar, Okhla, and parts of East Delhi were particularly hard-hit, with homes and apartment complexes submerged under several feet of water. The historic areas around Chandni Chowk and Connaught Place also saw significant water accumulation, affecting businesses and leading to substantial economic losses.

Gurgaon, situated just southwest of Delhi, also faced the brunt of the torrential rains. Although the city was not as severely affected as Delhi, several areas in Gurgaon experienced significant waterlo

In [ ]:
def get_coordinates(query):
  url = f"https://api.geoapify.com/v1/geocode/search?text={query}&lang=en&limit=3&filter=countrycode:in&format=json&apiKey={api_key}"

  headers = CaseInsensitiveDict()
  headers["Accept"] = "application/json"

  resp = requests.get(url, headers=headers)

  return resp.json()['results']

coord = []
res = []

for query in data:
  doc = ner(query)
  for ent in doc.ents:
    if ent.label_ == "GPE" or ent.label_ == "LOC":
      res = get_coordinates(ent.text)
      for r in res:
        arr = [r['lon'], r['lat']]
        coord.append([query, arr])

target = coordinates['Delhi']


for loc in coord:
  dist = manhattan_distances([target], [loc[1]])

  if dist < 5:
    if loc[0] not in res:
      res.append(loc[0])